In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
import statistics
from get_bounds import *
from k_means_equal_size import *
from sklearn.cluster import KMeans

### Grid Points Load (output from scoring.ipynb)
#### Grid Points are now scored in `scoring.ipynb`.

In [ ]:
YYZ_GridPoints = pd.read_csv('../res/grid_points/yyz_grid_points.csv', sep = ',')

### Parameters

In [ ]:
STD_DEV = 0.6
PT_DENSITY = 50

### Regression

In [ ]:
def linearRegression(scores):
    # step 1: don't use points that are very small (threshold)
    threshold = statistics.median(scores['score'])
    thresholded_scores = scores.loc[scores['score'] >= threshold]
    #print(thresholded_scores)
    
    regr = LinearRegression()
    lats = thresholded_scores["lat"].values.reshape(-1, 1)
    longs = thresholded_scores["long"].values.reshape(-1, 1)
    regr.fit(lats, longs, thresholded_scores["score"]) # Score is Squared because sample_weight is square rooted thresholded_scores["score"]
    x_begin = lats[0]
    x_end = lats[-1]
    y_begin = regr.predict(x_begin.reshape(1, -1)).item()
    y_end = regr.predict(x_end.reshape(1, -1)).item()
    
    return (x_begin.item(), x_end.item(), y_begin, y_end)

In [ ]:
def polynomialRegression(scores, degree, upperLatBound, bottomLatBound):
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_features = poly.fit_transform(scores["lat"].values.reshape(-1, 1))
    poly_reg_model = LinearRegression().fit(poly_features, scores["long"], scores["score"])
    x = scores['lat'].unique()
    #x = np.arange(bottomLatBound, upperLatBound, round((upperLatBound-bottomLatBound)/PT_DENSITY, 4))
    y = poly_reg_model.predict(poly.fit_transform(x.reshape(-1, 1)))
    return (x, y, poly_reg_model.intercept_, poly_reg_model.coef_)

### Filtering and Processing

In [ ]:
def trimOutOfBounds(points, upperLatBound, bottomLatBound, leftLongBound, rightLongBound):
    df = pd.DataFrame({'lat': points[0], 'long': points[1]})
    df = df.loc[(df['lat'] >= bottomLatBound) & (df['lat'] <= upperLatBound) & (df['long'] >= leftLongBound) & (df['long'] <= rightLongBound)]
    return (df['lat'].values, df['long'].values)

In [ ]:
def snapLongToGrid(points, leftLongBound, rightLongBound):
    step = round((rightLongBound-leftLongBound)/(PT_DENSITY-1), 4)
    pointsDf = pd.DataFrame({'lat': points[0], 'long': points[1]})
    for index, row in pointsDf.iterrows():
        pointsDf.at[index, 'long'] = pointsDf.at[index, 'long'] - ((pointsDf.at[index, 'long'] - leftLongBound) % step)
    return (pointsDf['lat'].values, pointsDf['long'].values)

### Plotting Functions

In [ ]:
def plotScores(scores):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    fig.set_figheight(10)
    fig.set_figwidth(10)
    ax.scatter(scores["lat"], scores["long"], scores["score"])
    return ax

In [ ]:
def plotScoresWithLine(scores, line):
    ax = plotScores(scores)
    ax.plot([line[0], line[1]], [line[2], line[3]], [0, 0], color='red', linewidth=3)
    return ax

In [ ]:
def plotScoresWithCurve(scores, curve):
    ax = plotScores(scores)
    ax.scatter(curve[0], curve[1], 0)
    return ax

In [ ]:
def plotHeatmap(scores, line):
    y_begin = line[2]
    y_end = line[3]
    fig = plt.figure()
    ax = sns.heatmap(scores.pivot("long", "lat", "score"))
    y_begin_hm = int((y_begin - scores["long"].iloc[0]) / (scores["long"].iloc[-1] - scores["long"].iloc[0]) * PT_DENSITY)
    y_end_hm = int((y_end - scores["long"].iloc[0]) / (scores["long"].iloc[-1] - scores["long"].iloc[0]) * PT_DENSITY)
    ax.plot([0, PT_DENSITY], [y_begin_hm, y_end_hm], linewidth=3, color='r')
    return ax

In [ ]:
def plotHeatmapPolynomial(scores, curves):
    fig = plt.figure()
    ax = sns.heatmap(scores.pivot("long", "lat", "score"))
    for curve in curves:
        x = curve[0]
        y = curve[1]
        x_hm = [int(a) for a in ((x - scores["lat"].iloc[0]) / (scores["lat"].iloc[-1] - scores["lat"].iloc[0]) * PT_DENSITY)]
        y_hm = [int(a) for a in ((y - scores["long"].iloc[0]) / (scores["long"].iloc[-1] - scores["long"].iloc[0]) * PT_DENSITY)]
        ax.scatter(x_hm, y_hm, linewidth=1)
    return ax

### Clustering Functions

### Toronto (YYZ)

In [ ]:
[YYZ_UpperLatBound, YYZ_BottomLatBound, YYZ_LeftLongBound, YYZ_RightLongBound] = getBounds(YYZ_GridPoints['lat'], YYZ_GridPoints['long'], 4)

### Clustering

In [ ]:
NUM_CLUSTERS = 6
INTERCHANGE_CANDIDATE_BONUS_FACTOR = 5

In [ ]:
YYZ_C_Scores = YYZ_GridPoints.copy()
YYZ_C_Scores = YYZ_C_Scores.loc[YYZ_C_Scores['score'] > 0.5]

In [ ]:
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=0).fit(YYZ_C_Scores[["lat", "long"]], 0, YYZ_C_Scores["score"])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
fig.set_figheight(10)
fig.set_figwidth(10)
ax.scatter(YYZ_C_Scores["lat"], YYZ_C_Scores["long"], YYZ_C_Scores["score"], c=kmeans.labels_.astype(float))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_figheight(10)
fig.set_figwidth(10)
ax.scatter(YYZ_C_Scores["lat"], YYZ_C_Scores["long"], c=kmeans.labels_.astype(float))

In [ ]:
YYZ_C_Scores['cluster'] = kmeans.labels_.tolist()

In [ ]:
# we have to sort the clusters so the downtown line gets created first
def sortClustersByScore(gridPoints):
    clusterScoreSums = []
    for i in range(NUM_CLUSTERS):
        selectedCluster = gridPoints.loc[gridPoints['cluster'] == i]
        clusterScoreSums.append(selectedCluster['score'].sum())
    newClusters = dict(enumerate(np.argsort(clusterScoreSums).flatten(), 0))
    gridPoints['cluster'] = gridPoints['cluster'].map(newClusters)
    return gridPoints

YYZ_C_Scores = sortClustersByScore(YYZ_C_Scores)

In [ ]:
def getLine(scores, upper, bottom, left, right):
    [*line, line_intercept, line_coefficients] = polynomialRegression(scores, 5, upper, bottom)
    line = trimOutOfBounds(line, upper, bottom, left, right)
    line = snapLongToGrid(line, left, right)
    return (line, line_intercept, line_coefficients)

In [ ]:
def getPointWithHighestScore(pts):
    return pts[pts.score == pts.score.max()]

In [ ]:
def getGridPointsFromLine(scores, line):
    numPoints = len(line[0])
    lineGridPoints = pd.DataFrame()

    for index in range(0, numPoints):
        lati = line[0][index]
        longi = line[1][index]

        scoreIndex = scores.loc[(abs(scores["lat"] - lati) <= 0.005) & (abs(scores["long"] - longi) <= 0.005)].index.values
        if len(scoreIndex) > 0:
            lineGridPoints = lineGridPoints.append(scores.iloc[scoreIndex[0]])

    return lineGridPoints

In [ ]:
YYZ_Cx_Scores = []
for i in range(NUM_CLUSTERS):
    YYZ_Cx_Scores.append(YYZ_C_Scores.loc[YYZ_C_Scores['cluster'] == i])

In [ ]:
YYZ_Cx_Line = []
YYZ_Cx_Interchange_Candidates = []
for i in range(NUM_CLUSTERS):
    YYZ_Cx_Scores[i] = YYZ_Cx_Scores[i].append(YYZ_Cx_Interchange_Candidates)
    [line, _, _] = getLine(YYZ_Cx_Scores[i], YYZ_UpperLatBound, YYZ_BottomLatBound, YYZ_LeftLongBound, YYZ_RightLongBound)
    YYZ_Cx_Line.append(line)
    linePoints = getGridPointsFromLine(YYZ_GridPoints, line)
    interchangeCandidate = getPointWithHighestScore(linePoints)
    interchangeCandidate = interchangeCandidate.assign(score=interchangeCandidate['score']*INTERCHANGE_CANDIDATE_BONUS_FACTOR)
    interchangeCandidate['external'] = True
    YYZ_Cx_Interchange_Candidates.append(interchangeCandidate)

    interchangeCandidatePoints = [(pd.concat(YYZ_Cx_Interchange_Candidates)["lat"].to_numpy(), pd.concat(YYZ_Cx_Interchange_Candidates)["long"].to_numpy())]
    plotHeatmapPolynomial(YYZ_GridPoints, YYZ_Cx_Line + interchangeCandidatePoints)

In [ ]:
YYZ_Cx_Line

In [ ]:
round((YYZ_RightLongBound-YYZ_LeftLongBound)/(PT_DENSITY - 1), 4)

In [ ]:
getGridPointsFromLine(YYZ_GridPoints, YYZ_Cx_Line[0])

In [ ]:
interchangeCandidatePoints = [(pd.concat(YYZ_Cx_Interchange_Candidates)["lat"].to_numpy(), pd.concat(YYZ_Cx_Interchange_Candidates)["long"].to_numpy())]
plotHeatmapPolynomial(YYZ_GridPoints, YYZ_Cx_Line + interchangeCandidatePoints)

### Export Cells

In [ ]:
with open('coords.txt', 'w') as f:
    f.write("----------LATS-------\n")
    for line in YYZ_Cx_Line:
        f.write("[")
        for lat in line[0]:
            f.write(str(lat))
            f.write(',')
        f.write("],")
        f.write("\n")
    f.write("----------LONGS-------\n")
    for line in YYZ_Cx_Line:
        f.write("[")
        for long in line[1]:
            f.write(str(long))
            f.write(',')
        f.write("],")
        f.write("\n")

# Finding Stations

In [ ]:
#line numbers are 0 indexed
YYZ_Cx_Station = []


def stationFinder(threshold, subwayLines, lineNum, YYZ_Cx_Station):
    size = len(subwayLines[lineNum][0])
    df = pd.DataFrame()

    for index in range(0, size):

        lati = subwayLines[lineNum][0][index]
        longi = subwayLines[lineNum][1][index]

        #print(YYZ_C_Scores.index.values)

        scoreIndex = YYZ_GridPoints.loc[(abs(YYZ_GridPoints["lat"] - lati) <= 0.00001) & (
                    abs(YYZ_GridPoints["long"] - longi) <= 0.00001)].index.values
        #print(scoreIndex)

        #check the score at this lat and long
        if len(scoreIndex) > 0:

            scoreInd = scoreIndex[0]
            score = YYZ_GridPoints["score"][scoreIndex]

            #print(score)

            #if above desire threshold add to data frame which we will then convert to numpy array
            if score[scoreIndex][scoreInd] >= threshold:
                #print("Hello")
                if df.empty:
                    d = {'lat': [lati], 'long': [longi]}
                    df = pd.DataFrame(data=d)
                else:

                    new_row = {'lat': lati, 'long': longi}
                    df = df.append(new_row, ignore_index=True)

    #convert data frame to numpy array to append to list so we can plot
    lineStat = (df['lat'].values, df['long'].values)

    YYZ_Cx_Station.append(lineStat)

    #return the new set of points
    return YYZ_Cx_Station

In [ ]:
#lineNum is not 0 indexed
def getStation(threshold, lineNum):
    for idx in range(0, lineNum):
        stationFinder(threshold,YYZ_Cx_Line, idx, YYZ_Cx_Station)
    #return YYZ_Cx_Station

In [ ]:
getStation(0.9, 5)

In [ ]:
plotHeatmapPolynomial(YYZ_GridPoints, YYZ_Cx_Station)

In [ ]:
json.dumps()